In [1]:
"""
NSE Option Chain Data Monitoring Script

This script fetches real-time option chain data from the NSE (National Stock Exchange)
and calculates the Put-Call Ratio (PCR) to determine market sentiment. It includes the following features:
- Real-time cookie handling for API authentication.
- Option chain data fetching with error handling.
- PCR calculation and sentiment classification.
- Optional Selenium integration for CAPTCHA handling.
- Automated scheduling for real-time data updates.

Author: Darshan Vinayak Shinde
Date: 21-01-2025
GitHub: https://github.com/darshanleads09

Prerequisites:
- Python 3.8 or higher
- Required libraries: requests, selenium, schedule
- ChromeDriver for Selenium
"""
import requests
import json
import time
import schedule
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from threading import Thread


# Function to fetch real-time cookies from NSE
def fetch_cookies():
    try:
        session = requests.Session()
        response = session.get("https://www.nseindia.com", headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        })
        response.raise_for_status()
        return session.cookies.get_dict()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching cookies: {e}")
        return None

# Function to fetch option chain data using the fetched cookies
def fetch_option_chain():
    cookies = fetch_cookies()
    if not cookies:
        print("Failed to fetch cookies. Cannot proceed.")
        return None

    headers = {
        "accept": "*/*",
        "accept-language": "en-US,en;q=0.9",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "referer": "https://www.nseindia.com/option-chain",
        "referrer-policy": "strict-origin-when-cross-origin",
    }

    url = "https://www.nseindia.com/api/option-chain-indices?symbol=BANKNIFTY"

    try:
        response = requests.get(url, headers=headers, cookies=cookies)
        response.raise_for_status()
        option_chain_data = response.json()
        return option_chain_data
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Connection Error: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Request Exception: {err}")
    return None

# Function to calculate PCR (Put-Call Ratio) from the option chain data
def calculate_pcr(option_chain_data):
    if not option_chain_data:
        print("No option chain data available.")
        return None

    try:
        puts_oi = sum(item['PE']['openInterest'] for item in option_chain_data['records']['data'] if 'PE' in item)
        calls_oi = sum(item['CE']['openInterest'] for item in option_chain_data['records']['data'] if 'CE' in item)
        pcr = puts_oi / calls_oi if calls_oi > 0 else 0
        return {
            "PCR": pcr,
            "Total Puts OI": puts_oi,
            "Total Calls OI": calls_oi
        }
    except KeyError as e:
        print(f"Error processing option chain data: {e}")
        return None

# Selenium setup for handling Captcha (if needed)
def handle_captcha_with_selenium():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_service = Service("chromedriver.exe")  # Adjust path as necessary
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

    try:
        driver.get("https://www.nseindia.com/option-chain")
        time.sleep(5)  # Wait for the page to load
        captcha_element = driver.find_element(By.CLASS_NAME, "some-captcha-class")
        if captcha_element:
            print("Captcha detected! Manual intervention required.")
            return None
        cookies = driver.get_cookies()
        driver.quit()
        return {cookie['name']: cookie['value'] for cookie in cookies}
    except Exception as e:
        print(f"Error handling captcha with Selenium: {e}")
        driver.quit()
        return None

# Main function to fetch and analyze option chain data
def main():
    print("==============================================================================================")
    print("Fetching real-time option chain data...")
    option_chain_data = fetch_option_chain()

    if option_chain_data:
        pcr_data = calculate_pcr(option_chain_data)
        if pcr_data:
            print(f"PCR (Put-Call Ratio): {pcr_data['PCR']}")
            print(f"Total Puts Open Interest: {pcr_data['Total Puts OI']}")
            print(f"Total Calls Open Interest: {pcr_data['Total Calls OI']}")

            # Determine sentiment based on PCR
            if pcr_data['PCR'] > 1.2:
                print("Market Sentiment: Bullish")
            elif pcr_data['PCR'] < 0.8:
                print("Market Sentiment: Bearish")
            else:
                print("Market Sentiment: Neutral")
        else:
            print("Failed to calculate PCR.")
    else:
        print("Failed to fetch option chain data.")
        
# Schedule the main function to run every minute
schedule.every(1).minutes.do(main)

# Start the real-time loop
print("Starting real-time monitoring...")
while True:
    schedule.run_pending()
    time.sleep(1)

Starting real-time monitoring...
Fetching real-time option chain data...
PCR (Put-Call Ratio): 0.5432989885447557
Total Puts Open Interest: 2114125.5
Total Calls Open Interest: 3891274.5
Market Sentiment: Bearish
Fetching real-time option chain data...
PCR (Put-Call Ratio): 0.5432989885447557
Total Puts Open Interest: 2114125.5
Total Calls Open Interest: 3891274.5
Market Sentiment: Bearish
Fetching real-time option chain data...
PCR (Put-Call Ratio): 0.5432311444489459
Total Puts Open Interest: 2113861.5
Total Calls Open Interest: 3891274.5
Market Sentiment: Bearish
Fetching real-time option chain data...
PCR (Put-Call Ratio): 0.5432989885447557
Total Puts Open Interest: 2114125.5
Total Calls Open Interest: 3891274.5
Market Sentiment: Bearish


KeyboardInterrupt: 